In [ ]:
# two-way table. Credits to Microsoft Copilot for most of the code.
# To do: add representation of Earth
# 2d: change size of dots, overlay earth map for reference
# Poster: add the starmaps first, with some labels.
import dash
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output, State, Dash, no_update
import plotly.graph_objects as go
import plotly.express as px
%run -i ../assets/lists.ipynb

# imports Planetary systems csv
pl_s = pd.read_csv('../assets/Planetary_Systems.csv', comment="#", low_memory=False)
pl_es = pd.read_csv('../assets/Planetary_Systems_Estimated.csv', comment="#", low_memory=False)

# imports atmospheric list for one filter.
at_s = pd.read_csv('../assets/Atmospheric_Spectroscopy.csv', comment="#", low_memory=False)

dropdown_options = [
    {'label': 'Planet Type', 'value': 'pl_type'},
    {'label': 'Harvard Spec. Class', 'value': 'st_teffclass'},
    {'label': 'Yerkes Spec. Class', 'value': 'st_lumclass'},
    {'label': 'Stellar Metallicity', 'value': 'st_metratio'},
    {'label': 'Discovery Method', 'value': 'discoverymethod'},
    {'label': 'Discovery Locale', 'value': 'disc_locale'},
    # {'label': 'Host Star', 'value': 'hostname'}, # too much computational power.
]

# Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Compare different exoplanet groupings"),

    html.H2("Data filters:"),
    dcc.Checklist(
        id='filter-checklist',
        options=[
            {'label': 'Has atmospheric data', 'value': 'atmoData'},
            {'label': 'Default parameter set', 'value': 'default'},
            {'label': 'No controversial flag', 'value': 'noControv'},
            {'label': 'Water to Metal Density', 'value': 'densRange'},
            {'label': 'In Target Star Catalog', 'value': 'target'},
        ],
        value=['default', 'noControv'],  # Default values to filter by
        className="checkbox-container"
    ),
    html.Div([
        dcc.Checklist(
            id='host-star-toggle',
            options=[{'label': 'Filter by host star: ', 'value': 'enable'}],
            value=[],  # Start unchecked
            className="checkbox-container"
        ),
        dcc.Input(
            id='host-star-input',
            type='text',
            placeholder='Enter host star name',
            debounce=True,
            className="textbox-style"
        ),
    ], className="checkbox-container", style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'gap': '10px'}),

    html.Details([
        html.Summary("Planet type"),
        dcc.Checklist(
            id='pltype-checklist',
            options=[
                {'label': 'Terrestrial', 'value': 'terrestrial'},
                {'label': 'Super-Earth', 'value': 'super_earth'},
                {'label': 'Neptune-like', 'value': 'neptune_like'},
                {'label': 'Gas Giant', 'value': 'gas_giant'},
                {'label': 'Unknown', 'value': 'unknown'},
                {'label': 'TBA', 'value': 'tba'},
            ],
            value=['terrestrial', 'super_earth', 'neptune_like', 'gas_giant', 'unknown', 'tba'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Discovery method"),
        dcc.Checklist(
            id='discmethod-checklist',
            options=[
                {'label': 'Transit', 'value': 'Transit'},
                {'label': 'Transit Timing Variations', 'value': 'Transit Timing Variations'},
                {'label': 'Eclipse Timing Variations', 'value': 'Eclipse Timing Variations'},
                {'label': 'Orbital Brightness Modulation', 'value': 'Orbital Brightness Modulation'},
                {'label': 'Radial Velocity', 'value': 'Radial Velocity'},
                {'label': 'Astrometry', 'value': 'Astrometry'},
                {'label': 'Imaging', 'value': 'Imaging'},
                {'label': 'Disc Kinematics', 'value': 'Disc Kinematics'},
                {'label': 'Microlensing', 'value': 'Microlensing'},
                {'label': 'Pulsar Timing', 'value': 'Pulsar Timing'},
                {'label': 'Pulsation Timing Variations', 'value': 'Pulsation Timing Variations'},
                {'label': 'Known Since Antiquity', 'value': 'Known Since Antiquity'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Transit', 'Radial Velocity', 'Imaging', 'Eclipse Timing Variations', 'Microlensing', 'Pulsar Timing', 'Pulsation Timing Variations', 'Orbital Brightness Modulation', 'Transit Timing Variations', 'Astrometry', 'Disc Kinematics', 'Known Since Antiquity', 'null'],  # automatically selects all
            className="checkbox-container"
        ),
    ]), 

    html.Details([
        html.Summary("Discovery locale"),
        dcc.Checklist(
            id='disclocale-checklist',
            options=[
                {'label': 'Space', 'value': 'Space'},
                {'label': 'Ground', 'value': 'Ground'},
                {'label': 'Multiple Locales', 'value': 'Multiple Locales'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['Space', 'Ground', 'Multiple Locales', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 
    
    html.Details([
        html.Summary("Harvard spectral classes"),
        dcc.Checklist(
            id='teff-checklist',
            options=[
                {'label': 'O type star (>33000 K)', 'value': 'O'},
                {'label': 'B type star (10000-33000 K)', 'value': 'B'},
                {'label': 'A type star (7300-10000 K)', 'value': 'A'},
                {'label': 'F type star (6000-7300 K)', 'value': 'F'},
                {'label': 'G type star (5300-6000 K)', 'value': 'G'},
                {'label': 'K type star (3900-5300 K, recommended)', 'value': 'K'},
                {'label': 'M type star (2300-3900 K)', 'value': 'M'},
                {'label': 'L type star (1300-2500 K)', 'value': 'L'},
                {'label': 'T type star (700-1300 K)', 'value': 'T'},
                {'label': 'Y type star (<700 K)', 'value': 'Y'},
                {'label': 'Wolf-Rayet star (>30000 K)', 'value': 'W'},
                {'label': 'White Dwarf (~5000-100000+ K)', 'value': 'D'},
                {'label': 'Unknown', 'value': 'null'}
            ],
            value=['O', 'B', 'A', 'F', 'G', 'K', 'M', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),

    html.Details([
        html.Summary("Yerkes spectral classes"),
        dcc.Checklist(
            id='lum-checklist',
            options=[
                # {'label': 'Hypergiant (0)', 'value': '0'},
                # {'label': 'Supergiant (I)', 'value': 'I'},
                {'label': 'Bright Giant (II)', 'value': 'II'},
                {'label': 'Giant (III)', 'value': 'III'},
                {'label': 'Subgiant (IV)', 'value': 'IV'},
                {'label': 'Main-sequence/Dwarf (V, recommended)', 'value': 'V'},
                {'label': 'Subdwarf (VI)', 'value': 'VI'},
                # {'label': 'White Dwarf (VII)', 'value': 'VII'},
                {'label': 'Unknown', 'value': 'null'}
            ],
            value=['0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]),
    
    html.Details([
        html.Summary("Stellar metallicity ratio"),
        dcc.Checklist(
            id='met-checklist',
            options=[
                {'label': 'Iron abundance (recommended)', 'value': '[Fe/H]'},
                {'label': 'General metal content', 'value': '[M/H]'},
                {'label': 'Unknown', 'value': 'null'},
            ],
            value=['[Fe/H]', '[M/H]', 'null'],  # Default values to filter by
            className="checkbox-container"
        ),
    ]), 

    html.H2("Group by:"),
    dcc.Dropdown(
        id='colorcode-dropdown-one',
        options=dropdown_options,
        value="pl_type",  # Default value
        className="dropdown-container",
        placeholder="First group"
    ),
    dcc.Dropdown(
        id='colorcode-dropdown-two',
        options=dropdown_options,
        value="discoverymethod",  # Default value
        className="dropdown-container",
        placeholder="Second group"
    ),
    
    html.H2("Extra options:"),
    dcc.Checklist(
        id='extra-checklist',
        options=[
            {'label': html.Span(['Simplify planet types and discovery methods'], style={'fontWeight': 'bold'}), 'value': 'simple'},
            {'label': html.Span(['Estimate missing values'], style={'fontWeight': 'bold'}), 'value': 'estimate'},
            # {'label': html.Span(['Compare stars'], style={'fontWeight': 'bold'}), 'value': 'suns'},
        ],
        value=[],  # Start unchecked
        className="checkbox-container"
    ),

    html.Button('Update Graph', id='update-button', n_clicks=1),

    dcc.Graph(id="table", figure={}, style={"display": "block"}),
    
    html.P('NaN info:', id='naninfo-box'),

    html.P("Data sourced from:"),
    html.Div([
        html.A("https://www.doi.org/10.26133/NEA12", href="https://www.doi.org/10.26133/NEA12", target="_blank"),
        html.Br(),
        html.A("https://www.doi.org/10.26133/NEA36", href="https://www.doi.org/10.26133/NEA36", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/exoplanet-catalog/", href="https://science.nasa.gov/exoplanets/exoplanet-catalog/", target="_blank"),
        html.Br(),
        html.A("https://science.nasa.gov/exoplanets/target-star-catalog/", href="https://science.nasa.gov/exoplanets/target-star-catalog/", target="_blank"),
    ]),

    dcc.Store(id='settings-store', storage_type='local'),
    dcc.Store(id='rehydration-complete', data=False),
], style={'color': 'black', 'font-family': 'Arial', 'backgroundColor': 'white', 'padding': '20px', 'text-align': 'center'})

def update_graph(update, filters, extra, startoggle, starinput, colorcodeone, colorcodetwo, pltype, discmethod, disclocale, teff, lum, met):
    # Load dataset
    if 'estimate' in extra:
        df = pl_es.copy()
    else:
        df = pl_s.copy()
    # print(df[colorcodeone].unique())
    # print(df[colorcodetwo].unique())
    # Apply filters
    if not 'suns' in extra:
        if 'atmoData' in filters:
            df = df[df['pl_name'].isin(at_s['pl_name']) | df["is_solar"] == True]
        if 'densRange' in filters:
            df=df[(df['pl_dens'] > 1) & (df['pl_dens'] < 5.6)] # Only gets planets with a density between water and metallic iron
        df = df[df['pl_type'].isin(pltype)]  
        if 'simple' in extra:
            df["pl_type"] = df["pl_type"].replace({"super_earth": "terrestrial", "neptune_like": "gas_giant"}) 
            df["discoverymethod"] = df["discoverymethod"].replace({"Transit Timing Variations": "Transit", "Eclipse Timing Variations": "Transit", "Orbital Brightness Modulation": "Transit", "Astrometry": "Radial Velocity", "Pulsation Timing Variations": "Pulsar Timing", "Disc Kinematics": "Imaging"}) 
        if 'null' in discmethod:
            df = df[df['discoverymethod'].isin(discmethod) | df['discoverymethod'].isna()]
        else:
            df = df[df['discoverymethod'].isin(discmethod)]
        if 'null' in disclocale:
            df = df[df['disc_locale'].isin(disclocale) | df['disc_locale'].isna()]
        else:
            df = df[df['disc_locale'].isin(disclocale)]
    
    if 'null' in teff:
        df = df[df['st_teffclass'].isin(teff) | df['st_teffclass'].isna()] 
    else:
        df = df[df['st_teffclass'].isin(teff)]   
    if 'null' in lum:
        df = df[df['st_lumclass'].isin(lum) | df['st_lumclass'].isna()] 
    else:
        df = df[df['st_lumclass'].isin(lum)]
    if 'default' in filters:
        df = df[df['default_flag'] == True]
    if 'noControv' in filters:
        df = df[df['pl_controv_flag'] == False]
    if 'target' in filters:
        df = df[df['hostname'].isin(target_stars) | df["is_solar"] == True] # type: ignore
    if 'null' in met:
        df = df[df['st_metratio'].isin(met) | df['st_metratio'].isna()]
    else:
        df = df[df['st_metratio'].isin(met)]
    if startoggle and starinput.strip():
        # Normalize both sides for safe matching
        df = df[df['hostname'].str.lower() == starinput.strip().lower()]

    # Returns empty graphs if filtered dataset is empty
    if df.empty:
        return go.Figure(),  "ERROR: filtered dataset is empty"
    # if "suns" in extra and (colorcodeone == "pl_type" or colorcodetwo == "pl_type"):
    #     return go.Figure(), "ERROR: Cannot filter by planet type when using star data."
    
    
    # Handle NaNs in colorcode by assigning 'unknown'
    df[colorcodeone] = df[colorcodeone].fillna("unknown")
    df[colorcodetwo] = df[colorcodetwo].fillna("unknown")

    table_data = pd.crosstab(df[colorcodeone], df[colorcodetwo])

    # Prepare headers and cells
    headers = ['Planet Type'] + list(table_data.columns)
    row_labels = list(table_data.index)
    cell_values = [row_labels] + [table_data[col].tolist() for col in table_data.columns]

    # def log_scaled_colors(cell_values, base_color="green", min_color="white"):
    #     colors = []
    #     for col in cell_values:
    #         col_colors = []
    #         numeric_vals = [v for v in col if isinstance(v, (int, float)) and v > 0]
    #         if not numeric_vals:
    #             colors.append(["white"] * len(col))
    #             continue

    #         log_vals = np.log10(np.array(numeric_vals))
    #         min_log, max_log = log_vals.min(), log_vals.max()

    #         for val in col:
    #             if val is None or val == 0:
    #                 col_colors.append(min_color)
    #             elif isinstance(val, (int, float)):
    #                 log_val = np.log10(val)
    #                 intensity = (log_val - min_log) / (max_log - min_log + 1e-9)
    #                 # Use a green gradient from white to dark green
    #                 r = int(255 * (1 - intensity))
    #                 g = int(255 * intensity)
    #                 b = int(255 * (1 - intensity))
    #                 col_colors.append(f"rgb({r},{g},{b})")
    #             else:
    #                 col_colors.append(min_color)
    #         colors.append(col_colors)
    #     return colors

    def get_cell_colors(cell_values, min_color="white"):
        flat_vals = [
            val for col in cell_values
            for val in col
            if isinstance(val, (int, float)) and not np.isnan(val)
        ]

        if not flat_vals:
            return [["white"] * len(col) for col in cell_values]

        min_val, max_val = min(flat_vals), max(flat_vals)
        range_val = max_val - min_val + 1e-9

        colors = []
        for col in cell_values:
            col_colors = []
            for val in col:
                if val is None or np.isnan(val):
                    col_colors.append(min_color)
                elif isinstance(val, (int, float)):
                    intensity = max(0, min(1, (val - min_val) / range_val))
                    r = g = int(255 * (1 - intensity))
                    b = 255
                    col_colors.append(f"rgb({r},{g},{b})")
                else:
                    col_colors.append(min_color)
            colors.append(col_colors)
        return colors


    
    # Create the Plotly table
    cell_colors = get_cell_colors(cell_values[1:])  # Skip header row
    fig = go.Figure(data=[go.Table(
        header=dict(values=headers, fill_color='lightblue', align='left'),
        # cells=dict(values=cell_values, fill_color=cell_colors, align='left')
        cells=dict(values=cell_values, fill_color="lavender", align='left')
    )])
    # cell_colors = log_scaled_colors(cell_values[1:])  # Skip header row

    return fig, "Rendered successfully."

@app.callback(
    [Output('table', 'figure'),
     Output('naninfo-box', 'children'),
     Output('settings-store', 'data')],
    [Input('update-button', 'n_clicks'),
     Input('settings-store', 'data')],
    [State('filter-checklist', 'value'),
     State('extra-checklist', 'value'),
     State('host-star-toggle', 'value'),
     State('host-star-input', 'value'),
     State('colorcode-dropdown-one', 'value'),
     State('colorcode-dropdown-two', 'value'),
     State('pltype-checklist', 'value'),
     State('discmethod-checklist', 'value'),
     State('disclocale-checklist', 'value'),
     State('teff-checklist', 'value'),
     State('lum-checklist', 'value'),
     State('met-checklist', 'value'),]
)
def update_figures(n_clicks, settings, *states):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    trigger = ctx.triggered[0]['prop_id']

    if trigger == 'update-button.n_clicks':
        # Use current inputs
        fig, naninfo = update_graph(n_clicks, *states)
        settings_dict = dict(zip([
            'filter', 'extra', 'hosttoggle', 'hostinput', 'colorcodeone', 'colorcodetwo',
            'pltype', 'discmethod', 'disclocale', 'teff', 'lum', 'met'
        ], states))
        return fig, naninfo, settings_dict

    elif trigger == 'settings-store.data':
        # Use stored settings, not states
        if not settings:
            raise dash.exceptions.PreventUpdate
        fig, naninfo = update_graph(0, *unpack_settings(settings))
        return fig, naninfo, dash.no_update

@app.callback(
    [Output('filter-checklist', 'value'),
     Output('extra-checklist', 'value'),
     Output('host-star-toggle', 'value'),
     Output('host-star-input', 'value'),
     Output('colorcode-dropdown-one', 'value'),
     Output('colorcode-dropdown-two', 'value'),
     Output('pltype-checklist', 'value'),
     Output('discmethod-checklist', 'value'),
     Output('disclocale-checklist', 'value'),
     Output('teff-checklist', 'value'),
     Output('lum-checklist', 'value'),
     Output('met-checklist', 'value'),],
    Input('settings-store', 'data'),
    prevent_initial_call=False
)
def restore_settings(settings):
    if not settings:
        raise dash.exceptions.PreventUpdate
    return (
        settings['filter'], settings['extra'], settings['hosttoggle'], settings['hostinput'],
        settings['colorcodeone'], settings['colorcodetwo'], settings['pltype'], settings['discmethod'],
        settings['disclocale'], settings['teff'], settings['lum'], settings['met'],
    )

def unpack_settings(settings):
    return (
        settings['filter'], settings['extra'], settings['hosttoggle'], settings['hostinput'],
        settings['colorcodeone'], settings['colorcodetwo'], settings['pltype'], settings['discmethod'],
        settings['disclocale'], settings['teff'], settings['lum'], settings['met'],
    )

if __name__ == '__main__':
    app.run(debug=True, port=8056)

    # get bargraphs with distribution of both terrestrials only and all planets

{'filter': ['default', 'noControv'], 'extra': ['estimate'], 'hosttoggle': [], 'hostinput': None, 'colorcodeone': 'st_teffclass', 'colorcodetwo': 'discoverymethod', 'pltype': ['terrestrial', 'super_earth', 'neptune_like', 'gas_giant', 'unknown', 'tba'], 'discmethod': ['Transit', 'Radial Velocity', 'Imaging', 'Eclipse Timing Variations', 'Microlensing', 'Pulsar Timing', 'Pulsation Timing Variations', 'Orbital Brightness Modulation', 'Transit Timing Variations', 'Astrometry', 'Disc Kinematics', 'Known Since Antiquity', 'null'], 'disclocale': ['Space', 'Ground', 'Multiple Locales', 'null'], 'teff': ['O', 'B', 'A', 'F', 'G', 'K', 'M', 'null'], 'lum': ['0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'null'], 'met': ['[Fe/H]', '[M/H]', 'null']}
